In [8]:
# Domino
from domino import embed
from domino._embed.encoder import Encoder
from domino import explore, DominoSlicer
# Standard library imports
import meerkat as mk
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import torch
from torchvision import transforms
import os
import csv
import pandas as pd
import numpy as np
# Clip model 
from open_clip import create_model_from_pretrained, get_tokenizer
from PIL import Image
import random
from meerkat import DataPanel
import pickle
from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas() 

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from utils import *
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
data_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/physionet.org/files/mimic-cxr-jpg/2.1.0/mimic-cxr-2.0.0-chexpert.csv")
# split the data into train and test sets by 0.8 and 0.2
# train_df = data_df.sample(frac=0.8, random_state=42).reset_index(drop=True)
# test_df = data_df.drop(train_df.index).reset_index(drop=True)   
# # Save the train and test DataFrames to CSV files
# # train_df.to_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df.csv", index=False)
# # test_df.to_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df.csv", index=False)
# Load the train and test DataFrames

# train_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df.csv")
# test_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df.csv")

data_df["No Finding"].value_counts()

No Finding
1.0    75455
Name: count, dtype: int64

Train finding & cardio

In [41]:
# All the No finding columns have been  set to 1 or 0
# clean the data_df on finding
# Fill null values in "No Finding" column with 0
data_df["No Finding"].fillna(0, inplace=True)
data_df_cleaned_finding = data_df[data_df["No Finding"].isin([1, 0])].reset_index(drop=True)
#data_df_cleaned_finding = data_df_cleaned_finding[data_df_cleaned_finding["Cardiomegaly"].isin([1, 0])].reset_index(drop=True)
print(f"Data cleaned on finding: {data_df_cleaned_finding.shape}")
# Split the data into train and test sets by 0.8 and 0.2
train_df = data_df_cleaned_finding.sample(frac=0.8, random_state=42).reset_index(drop=True)
train_df = train_df[train_df["Cardiomegaly"].isin([1, 0])].reset_index(drop=True).copy()
test_df = data_df_cleaned_finding.drop(train_df.index).reset_index(drop=True)   
# Save the train and test DataFrames to CSV files
print(f" Cardiomegaly: {train_df['Cardiomegaly'].value_counts()}")
print(f" Cardiomegaly: {test_df['Cardiomegaly'].value_counts()}")
# flip the No Finding column to Finding column
train_df['Finding'] = train_df['No Finding'].apply(lambda x: 0 if x == 1 else 1)

# Save the files 
train_df.to_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df_findings_cardio.csv", index=False)
#test_df.to_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_findings_cardio.csv", index=False)




/tmp/ipykernel_3251480/2005903429.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df["No Finding"].fillna(0, inplace=True)


Data cleaned on finding: (227827, 16)
 Cardiomegaly: Cardiomegaly
1.0    35840
0.0    12743
Name: count, dtype: int64
 Cardiomegaly: Cardiomegaly
 1.0    34884
 0.0    12646
-1.0     4740
Name: count, dtype: int64


In [39]:
train_df.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,Finding
0,10335334,56048989,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,1.0,1
1,13007347,51404713,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.0,1.0,0
2,16425412,58032996,1.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1
3,15281216,59204145,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,1
4,17419409,54829080,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,1.0,0.0,NaN,0.0,NaN,NaN,0


In [6]:
test_df_new = pd.read_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/test_df_with_embeddings.parquet")

In [42]:
test_df_new.head()
test_df_new['No Finding'].fillna(0, inplace=True)
test_df_new = test_df_new[test_df_new["No Finding"].isin([1, 0])].reset_index(drop=True)
test_df_new['Cardiomegaly'].value_counts()
#len(test_df_new)
# fill all cardiomegaly values if not 1 than set to 0
test_df_new['Cardiomegaly'].fillna(0, inplace=True)
test_df_new = test_df_new[test_df_new["Cardiomegaly"].isin([1, 0])].reset_index(drop=True)
print(f"Data cleaned on finding: {test_df_new.shape}")
# Save the test DataFrame to CSV file
#test_df_new.to_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_findings_cardio.csv", index=False)
test_df_new['Finding'] = test_df_new['No Finding'].apply(lambda x: 0 if x == 1 else 1)
test_df_new.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_with_embeddings_cardio.parquet", index=False)

/tmp/ipykernel_3251480/363068135.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df_new['No Finding'].fillna(0, inplace=True)
/tmp/ipykernel_3251480/363068135.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

Data cleaned on finding: (7946, 40)


Train No Finding & View Position 

In [51]:
data_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/physionet.org/files/mimic-cxr-jpg/2.1.0/mimic-cxr-2.0.0-chexpert.csv")
data_df["No Finding"].fillna(0)
data_df_cleaned_finding = data_df[data_df["No Finding"].isin([1, 0])].reset_index(drop=True)
meta_df = pd.read_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/meta_df_processed.parquet")

# Merge the data_df_cleaned_finding to meta_df_processed on "subject_id" and "study_id"
data_meta_df = pd.merge(data_df_cleaned_finding, meta_df, on=["subject_id", "study_id"], how="right")
data_meta_df['view_position_text'].value_counts()
train_df = data_meta_df.sample(frac=0.8, random_state=42).reset_index(drop=True)
# drop all the rows with view_position_text as "standard view'
train_df = train_df[train_df["view_position_text"] != "standard view"].reset_index(drop=True)
train_df['Position'] = [0 if 'lateral' in x else 1 for x in train_df['view_position_text']]
train_df['Position'].value_counts()
train_df['Finding'] = train_df['No Finding'].apply(lambda x: 0 if x == 1 else 1)
train_df.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df_position.parquet", index=False)




In [55]:
train_df.columns

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows',
       'Columns', 'StudyDate', 'StudyTime',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'id', 'exam_type_text',
       'view_position_text', 'patient_orientation_text', 'image_quality_text',
       'study_time_text', 'metadata_description', 'Position', 'Finding'],
      dtype='object')

In [52]:
meta_df.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,id,exam_type_text,view_position_text,patient_orientation_text,image_quality_text,study_time_text,metadata_description
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,"[10000032, 50414267]",chest radiography,posterior-anterior view,upright position,high-resolution,in May 2180 during evening hours,Patient underwent chest radiography in May 218...
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,"[10000032, 50414267]",chest radiography,lateral view,upright position,high-resolution,in May 2180 during evening hours,Patient underwent chest radiography in May 218...
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,"[10000032, 53189527]",chest radiography,posterior-anterior view,upright position,high-resolution,in June 2180 during afternoon hours,Patient underwent chest radiography in June 21...
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,"[10000032, 53189527]",chest radiography,lateral view,upright position,high-resolution,in June 2180 during afternoon hours,Patient underwent chest radiography in June 21...
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,None,"[10000032, 53911762]",bedside portable chest radiography,anterior-posterior view,upright position,standard resolution,in July 2180 during night hours,Patient underwent bedside portable chest radio...


In [60]:
print(test_meta_df.columns)

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'predicted', 'true', 'id_x', 'path', 'dicom_id_x',
       'PerformedProcedureStepDescription_x', 'ViewPosition_x', 'Rows_x',
       'Columns_x', 'StudyDate_x', 'StudyTime_x',
       'ProcedureCodeSequence_CodeMeaning_x', 'ViewCodeSequence_CodeMeaning_x',
       'PatientOrientationCodeSequence_CodeMeaning_x', 'exam_type_text_x',
       'view_position_text_x', 'patient_orientation_text_x',
       'image_quality_text_x', 'study_time_text_x', 'metadata_description_x',
       'report_text', 'image_embedding', 'report_embedding',
       'metadata_embedding', 'dicom_id_y',
       'PerformedProcedureStepDescription_y', 'ViewPosition_y', 'Rows_y',
       'Columns_y', 'StudyDate_y', 'StudyTime_y',
       'Proce

In [62]:
test_df_new = pd.read_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/test_df_with_embeddings.parquet")


test_df_new['No Finding'] = test_df_new['No Finding'].fillna(0)
test_df_new = test_df_new[test_df_new['No Finding'].isin([0, 1])].reset_index(drop=True).copy()



test_meta_df = test_df_new.copy()


test_meta_df['Finding'] = test_meta_df['No Finding'].apply(lambda x: 0 if x == 1 else 1)
test_meta_df['Position'] = [0 if 'lateral' in x else 1 for x in test_meta_df['view_position_text']]

print(test_meta_df['Position'].value_counts())
print(test_meta_df['Finding'].value_counts())


test_meta_df.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_position.parquet", index=False)

Position
1    7392
0    1005
Name: count, dtype: int64
Finding
1    7523
0     874
Name: count, dtype: int64


In [57]:
test_meta_df.columns

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'predicted', 'true', 'id_x', 'path', 'dicom_id_x',
       'PerformedProcedureStepDescription_x', 'ViewPosition_x', 'Rows_x',
       'Columns_x', 'StudyDate_x', 'StudyTime_x',
       'ProcedureCodeSequence_CodeMeaning_x', 'ViewCodeSequence_CodeMeaning_x',
       'PatientOrientationCodeSequence_CodeMeaning_x', 'exam_type_text_x',
       'view_position_text_x', 'patient_orientation_text_x',
       'image_quality_text_x', 'study_time_text_x', 'metadata_description_x',
       'report_text', 'image_embedding', 'report_embedding',
       'metadata_embedding', 'dicom_id_y',
       'PerformedProcedureStepDescription_y', 'ViewPosition_y', 'Rows_y',
       'Columns_y', 'StudyDate_y', 'StudyTime_y',
       'Proce

In [47]:
train_df['Position'].value_counts()

Position
1    64965
0    45001
Name: count, dtype: int64

### Metadata Preparation

In [14]:

meta_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/physionet.org/files/mimic-cxr-jpg/2.1.0/mimic-cxr-2.0.0-metadata.csv")
meta_df['id'] = list(zip(meta_df['subject_id'].astype(str), 
                            meta_df['study_id'].astype(str))) 
# Convert the information to the text format 
meta_df_processed = convert_medical_metadata_to_text(meta_df)  
meta_df_processed.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/meta_df_processed.parquet", index=False)


In [12]:
meta_df_processed.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,id,exam_type_text,view_position_text,patient_orientation_text,image_quality_text,study_time_text,metadata_description
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,"(10000032, 50414267)",chest radiography,posterior-anterior view,upright position,high-resolution,in May 2180 during evening hours,Patient underwent chest radiography in May 218...
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,"(10000032, 50414267)",chest radiography,lateral view,upright position,high-resolution,in May 2180 during evening hours,Patient underwent chest radiography in May 218...
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,"(10000032, 53189527)",chest radiography,posterior-anterior view,upright position,high-resolution,in June 2180 during afternoon hours,Patient underwent chest radiography in June 21...
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,"(10000032, 53189527)",chest radiography,lateral view,upright position,high-resolution,in June 2180 during afternoon hours,Patient underwent chest radiography in June 21...
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,"(10000032, 53911762)",bedside portable chest radiography,anterior-posterior view,upright position,standard resolution,in July 2180 during night hours,Patient underwent bedside portable chest radio...


### Metadata Preparation

In [11]:
meta_df_processed.head()
meta_df_processed["view_position_text"].value_counts()

view_position_text
anterior-posterior view        147173
lateral view                   117986
posterior-anterior view         96161
standard view                   15784
left anterior oblique view          3
right anterior oblique view         3
Name: count, dtype: int64

### Test Preprocessing 

Report Extraction

In [13]:
tqdm.pandas()
REPORTS_DIR = "/vol/bitbucket/yl28218/thesis/physionet.org/files/mimic-cxr-jpg/2.1.0/files/mimic_reports/"

def read_mimic_report(subject_id, study_id):
    subject_dir = f"p{str(subject_id)[:2]}"
    subject_full = f"p{subject_id}"
    report_file = f"s{study_id}.txt"
    file_path = os.path.join(REPORTS_DIR, 'files', subject_dir, subject_full, report_file)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except FileNotFoundError:
        return None


test_df['report_text'] = test_df.progress_apply(
    lambda row: read_mimic_report(row['subject_id'], row['study_id']),
    axis=1
)

100%|██████████| 10543/10543 [00:12<00:00, 843.08it/s] 


In [21]:
# import the path file
path = pickle.load(open('file_map_cache_complete.pkl', 'rb'))

# convert the path to a DataFrame
test_df['id'] = list(zip(test_df['subject_id'].astype(str), 
                         test_df['study_id'].astype(str)))
path_images = [k[0] for k in path.values()]
path_df = pd.DataFrame({"id": list(path.keys()), "path": path_images})
test_df = pd.merge(test_df, path_df, on='id', how='left')

# find the id without a path
missing_paths = test_df[test_df['path'].isnull()]['id'].tolist()
print(f"Number of missing paths: {len(missing_paths)}")


test_df = test_df[test_df['path'].notnull()].copy()

# extract dicom_id from path safely
test_df["dicom_id"] = test_df["path"].apply(lambda x: os.path.basename(x).split('.')[0])

# merge the test_df with the metadata
test_df = pd.merge(test_df, meta_df_processed, on='dicom_id', how='left')

# find the id without a metadata description
missing_metadata = test_df[test_df['metadata_description'].isnull()]['dicom_id'].tolist()
print(f"Number of missing metadata descriptions: {len(missing_metadata)}")

print("length of test_df:", len(test_df))


Number of missing paths: 1356
Number of missing metadata descriptions: 0
length of test_df: 9187


### Extract Embedding

In [ ]:
# Get the embedding for the images and report text
model, preprocess = create_model_from_pretrained(
    'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
)
tokenizer = get_tokenizer(
    'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device).eval()

# define Transform which resize the image to 224x224 and normalize it
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(           
        mean=[0.48145466, 0.4578275, 0.40821073],
        std=[0.26862954, 0.26130258, 0.27577711]
    )
])

def get_text_embedding(text):
    tokenized = tokenizer([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(tokenized)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)  
    return text_features[0].cpu().numpy() # Add batch dimension for consistency

def get_image_embedding(image_path):

    image = Image.open(image_path).convert('RGB')
    

    image_tensor = preprocess(image).unsqueeze(0).to(device) 
    
    with torch.no_grad():

        image_features = model.encode_image(image_tensor)

        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    return image_features[0].cpu().numpy()

In [ ]:
test_df['image_embedding'] = test_df['path'].progress_apply(get_image_embedding)
test_df["report_embedding"] = test_df['report_text'].progress_apply(get_text_embedding)
test_df["metadata_embedding"] = test_df['metadata_description'].progress_apply(get_text_embedding)
test_df.to_parquet('test_df_with_embeddings_v2.parquet', index=False)

In [6]:
train_df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,15517908,54944476,1.0,1.0,NaN,-1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,0.0,NaN
1,14997223,56919551,1.0,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
2,15831913,55952386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
3,18994929,53818785,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
4,12868814,58575191,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42166,14325424,58251915,NaN,NaN,1.0,NaN,-1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0
42167,13295971,50868944,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,1.0
42168,18527164,50617229,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0.0,1.0
42169,16898052,54026371,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0


### Train data set for meta classifiet

In [ ]:
train_meta_df = train_df.merge(
    meta_df_processed[['subject_id', 'study_id', 'exam_type_text']],
    on=['subject_id', 'study_id'],
    how='left'
)
train_meta_df['is_portable'] = train_meta_df['exam_type_text']\
    .str.contains('portable', case=False, na=False)



In [10]:
# convert to 1,0
train_meta_df['is_portable'] = train_meta_df['is_portable'].astype(int)

# test set 
test_df = pd.read_parquet('test_df_with_embeddings.parquet')

In [ ]:
test_meta_df = test_df.copy()
test_meta_df['is_portable'] = test_meta_df['exam_type_text']\
    .str.contains('portable', case=False, na=False)



In [20]:
train_meta_df = train_meta_df[train_meta_df['Cardiomegaly'].isin([0, 1])]
print("Train meta DataFrame shape:", train_meta_df.shape)
# value counts for Cardiomegaly
print(train_meta_df['Cardiomegaly'].value_counts())
test_meta_df = test_meta_df[test_meta_df['Cardiomegaly'].isin([0, 1])]
print("Test meta DataFrame shape:", test_meta_df.shape)
# value counts for Cardiomegaly
print(test_meta_df['Cardiomegaly'].value_counts())
# for portable value counts
print("Train meta DataFrame is_portable value counts:")
print(train_meta_df['is_portable'].value_counts())
print("Test meta DataFrame is_portable value counts:")
print(test_meta_df['is_portable'].value_counts())

Train meta DataFrame shape: (25239, 18)
Cardiomegaly
1.0    16597
0.0     8642
Name: count, dtype: int64
Test meta DataFrame shape: (3701, 41)
Cardiomegaly
1.0    2505
0.0    1196
Name: count, dtype: int64
Train meta DataFrame is_portable value counts:
is_portable
1    16016
0     9223
Name: count, dtype: int64
Test meta DataFrame is_portable value counts:
is_portable
True     2835
False     866
Name: count, dtype: int64


In [21]:
# save the train and test dataframes with meta embeddings 
train_meta_df.to_parquet('train_meta_df_with_embeddings.parquet', index=False)
test_meta_df.to_parquet('test_meta_df_with_embeddings.parquet', index=False)



In [22]:
data_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/physionet.org/files/mimic-cxr-jpg/2.1.0/mimic-cxr-2.0.0-chexpert.csv")
#data_df["No Finding"].fillna(0)
data_df["Cardiomegaly"].fillna(0, inplace=True)
data_df_cleaned_finding = data_df[data_df["Cardiomegaly"].isin([1, 0])].reset_index(drop=True)
meta_df = pd.read_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/meta_df_processed.parquet")

# Merge the data_df_cleaned_finding to meta_df_processed on "subject_id" and "study_id"
data_meta_df = pd.merge(data_df_cleaned_finding, meta_df, on=["subject_id", "study_id"], how="right")
data_meta_df['view_position_text'].value_counts()
train_df = data_meta_df.sample(frac=0.8, random_state=42).reset_index(drop=True)
# drop all the rows with view_position_text as "standard view'
train_df = train_df[train_df["view_position_text"] != "standard view"].reset_index(drop=True)
train_df['Position'] = [0 if 'lateral' in x else 1 for x in train_df['view_position_text']]
train_df['Cardiomegaly'].value_counts()

#train_df['Finding'] = train_df['No Finding'].apply(lambda x: 0 if x == 1 else 1)
#train_df.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df_position.parquet", index=False)

/tmp/ipykernel_1614328/2254601363.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df["Cardiomegaly"].fillna(0, inplace=True)


Cardiomegaly
0.0    233213
1.0     49289
Name: count, dtype: int64

In [25]:
#test_results.groupby(['Position', 'true']).size().unstack(fill_value=0)
train_df.groupby(['Position', 'Cardiomegaly']).size().unstack(fill_value=0)

Cardiomegaly,0.0,1.0
Position,,
0,81978,11157
1,151235,38132


In [26]:
# read the train and test dataframes with meta embeddings
test_df = pd.read_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_position.parquet")

In [28]:
test_df['Cardiomegaly'].fillna(0, inplace=True)
test_df = test_df[test_df["Cardiomegaly"].isin([1, 0])].reset_index(drop=True)
test_df.groupby(['Position', 'Cardiomegaly']).size().unstack(fill_value=0)
# Save the test DataFrame to CSV file

/tmp/ipykernel_1614328/3084683551.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Cardiomegaly'].fillna(0, inplace=True)


Cardiomegaly,0.0,1.0
Position,,
0,732,216
1,4709,2289


In [29]:
# save the dataframes with embeddings
test_df.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/test_df_with_embeddings_cardio_position.parquet", index=False)
train_df.to_parquet("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df_cardio_position.parquet", index=False)

### Create the embedding use clip

In [2]:
test_df = pd.read_parquet('test_df_with_embeddings.parquet')
train_df = pd.read_csv("/vol/bitbucket/yl28218/thesis/mimic_cxr_exp/data/train_df.csv")

In [10]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

def get_text_embedding(text):
    inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    return text_features[0].cpu().numpy()

def get_image_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=None, images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    return image_features[0].cpu().numpy()


/vol/bitbucket/yl28218/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
test_df['image_embedding'] = test_df['path'].progress_apply(get_image_embedding)


  0%|          | 0/8397 [00:00<?, ?it/s]

  0%|          | 0/8397 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (120 > 77). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (120) must match the size of tensor b (77) at non-singleton dimension 1

In [11]:
test_df["report_embedding"] = test_df['report_text'].progress_apply(get_text_embedding)


  0%|          | 0/8397 [00:00<?, ?it/s]

In [12]:
test_df["metadata_embedding"] = test_df['metadata_description'].progress_apply(get_text_embedding)
test_df.to_parquet('data/test_df_with_embeddings_clip.parquet', index=False)

  0%|          | 0/8397 [00:00<?, ?it/s]